In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf

import math
import matplotlib.pyplot as plt
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle
import gzip
import tensorflow as tf
# tf.random.set_seed(1004)
tf.random.set_seed(2)

gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        for i in range(len(gpus)):
            tf.config.experimental.set_memory_growth(gpus[i], True)
    except RuntimeError as e:
        # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


# Data Load

In [2]:
import random  
seed_num = 42
random.seed(seed_num)

x = np.load('/project/LSH/x_(7727,10,4068).npy')
y = np.load('/project/LSH/y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:,:], y[idx[:i]]
X_test, y_test = x[idx[i:],:,:], y[idx[i:]]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6182, 10, 4068), (6182,), (1545, 10, 4068), (1545,))

In [3]:
def MAE(y_true, y_pred):
    return "{:.2e}".format(np.mean(np.abs(y_true-y_pred)))

def MSE(y_true, y_pred):
    return "{:.2e}".format(np.mean(np.square(y_true-y_pred)))

def RMSE(y_true, y_pred):
    return "{:.2e}".format(np.sqrt(np.mean(np.square(y_true-y_pred))))

In [4]:
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.ensemble import VotingClassifier
from keras.layers import InputLayer, GRU
import keras

# Modeling

## LSTM 정의

In [5]:
tf.random.set_seed(seed_num)
# def get_model():
#     lstm = Sequential()
#     lstm.add(InputLayer(input_shape=(x.shape[1],x.shape[2])))
#     lstm.add(LSTM(units=128, activation='hard_sigmoid', return_sequences=True))
#     lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
#     lstm.add(Dropout(0.2))
#     lstm.add(LSTM(units=64, activation='hard_sigmoid', return_sequences=True))
#     lstm.add(LSTM(units=32, activation='hard_sigmoid', return_sequences=False))
#     lstm.add(Dropout(0.2))
#     lstm.add(Dense(units=1, activation='sigmoid'))
#     lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
#                           loss = "binary_crossentropy", metrics=['acc'])
#     return lstm
def get_model():
    lstm = Sequential()
    lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
    lstm.add(LSTM(units=128, activation='relu', return_sequences=True, kernel_regularizer=regularizers.l2(0.001)))
    lstm.add(LSTM(units=64, activation='relu', return_sequences=True, kernel_regularizer=regularizers.l2(0.001)))
    lstm.add(LSTM(units=32, activation='tanh', return_sequences=False, kernel_regularizer=regularizers.l2(0.001)))
    lstm.add(Dense(units=1))

    lstm.compile(optimizer= keras.optimizers.Adam(learning_rate = 0.001), 
                          loss = "binary_crossentropy", metrics=['acc'])
    return lstm

In [6]:
# lstm.fit(X_train,y_train, epochs=1000, batch_size=256, validation_split = 0.2)

# #예측 후 가공
# preds = lstm.predict(X_test)

# preds[preds>0.5]=1
# preds[preds<=0.5]=0

# from sklearn import metrics 
# print('정확도 :', metrics.accuracy_score(y_test, preds))
# print("RMSE : ", RMSE(y_test, preds))

## VotingClassifier 적용

In [15]:
%%time
#LSTM 쌓기
with tf.device('/device:GPU:1'):
    estimator = []
    for i in range(1,51):
        LSTM_Predictors = KerasClassifier(build_fn=lambda:get_model(), epochs=100, batch_size=256)
        LSTM_Predictors._estimator_type="classifier"
        estimator.append((f'model{i}', LSTM_Predictors))
    print(estimator)    

    final_model = VotingClassifier(estimators = estimator, voting = 'soft')
    final_model.fit(X_train,y_train)

    #예측 후 가공
    preds = final_model.predict(X_test)

    preds[preds>0.5]=1
    preds[preds<=0.5]=0

    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    roc_auc = roc_auc_score(y_test, preds)
    acc = accuracy_score(y_test, preds)

    print(f' accuracy : {acc}, precision : {precision}, recall : {recall}, f1 : {f1}, roc_auc : {roc_auc}')

[('model1', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f3800590b50>), ('model2', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f38004b8280>), ('model3', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f38004b8250>), ('model4', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f38004b8220>), ('model5', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f38004b8790>), ('model6', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f38004b8310>), ('model7', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f38004b8370>), ('model8', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f38004b8340>), ('model9', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7f38004b8c10>), ('model10', <tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object

25/25 [==============================] - 1s 29ms/step - loss: 0.3783 - acc: 0.9329
Epoch 30/100
25/25 [==============================] - 1s 29ms/step - loss: 0.5017 - acc: 0.8766
Epoch 31/100
25/25 [==============================] - 1s 29ms/step - loss: 0.3891 - acc: 0.9340
Epoch 32/100
25/25 [==============================] - 1s 29ms/step - loss: 0.3429 - acc: 0.9471
Epoch 33/100
25/25 [==============================] - 1s 30ms/step - loss: 0.3271 - acc: 0.9478
Epoch 34/100
25/25 [==============================] - 1s 29ms/step - loss: 0.3146 - acc: 0.9534
Epoch 35/100
25/25 [==============================] - 1s 30ms/step - loss: 0.3179 - acc: 0.9537
Epoch 36/100
25/25 [==============================] - 1s 29ms/step - loss: 0.3240 - acc: 0.9529
Epoch 37/100
25/25 [==============================] - 1s 29ms/step - loss: 0.6726 - acc: 0.8983
Epoch 38/100
25/25 [==============================] - 1s 29ms/step - loss: 0.4199 - acc: 0.9313
Epoch 39/100
25/25 [==============================] -

25/25 [==============================] - 1s 31ms/step - loss: 0.5482 - acc: 0.8687
Epoch 12/100
25/25 [==============================] - 1s 30ms/step - loss: 0.5336 - acc: 0.8895
Epoch 13/100
25/25 [==============================] - 1s 30ms/step - loss: 0.9764 - acc: 0.8418
Epoch 14/100
25/25 [==============================] - 1s 30ms/step - loss: 0.7035 - acc: 0.8195
Epoch 15/100
25/25 [==============================] - 1s 30ms/step - loss: 0.5757 - acc: 0.8675
Epoch 16/100
25/25 [==============================] - 1s 30ms/step - loss: 0.5392 - acc: 0.8897
Epoch 17/100
25/25 [==============================] - 1s 30ms/step - loss: 0.4953 - acc: 0.9080
Epoch 18/100
25/25 [==============================] - 1s 30ms/step - loss: 0.4484 - acc: 0.9249
Epoch 19/100
25/25 [==============================] - 1s 30ms/step - loss: 0.4261 - acc: 0.9329
Epoch 20/100
25/25 [==============================] - 1s 30ms/step - loss: 0.4165 - acc: 0.9355
Epoch 21/100
25/25 [==============================] -

25/25 [==============================] - 1s 27ms/step - loss: 0.3479 - acc: 0.9634
Epoch 97/100
25/25 [==============================] - 1s 28ms/step - loss: 0.3233 - acc: 0.9696
Epoch 98/100
25/25 [==============================] - 1s 28ms/step - loss: 0.2938 - acc: 0.9743
Epoch 99/100
25/25 [==============================] - 1s 27ms/step - loss: 0.2803 - acc: 0.9782
Epoch 100/100
25/25 [==============================] - 1s 27ms/step - loss: 0.2680 - acc: 0.9809
Epoch 1/100
25/25 [==============================] - 3s 29ms/step - loss: 1.8649 - acc: 0.5147
Epoch 2/100
25/25 [==============================] - 1s 30ms/step - loss: 0.9698 - acc: 0.6194
Epoch 3/100
25/25 [==============================] - 1s 30ms/step - loss: 0.8130 - acc: 0.6902
Epoch 4/100
25/25 [==============================] - 1s 30ms/step - loss: 0.7373 - acc: 0.7378
Epoch 5/100
25/25 [==============================] - 1s 29ms/step - loss: 0.6758 - acc: 0.7724
Epoch 6/100
25/25 [==============================] - 1s 2

25/25 [==============================] - 1s 29ms/step - loss: 0.2486 - acc: 0.9838
Epoch 79/100
25/25 [==============================] - 1s 29ms/step - loss: 0.2585 - acc: 0.9775
Epoch 80/100
25/25 [==============================] - 1s 28ms/step - loss: 0.2450 - acc: 0.9786
Epoch 81/100
25/25 [==============================] - 1s 29ms/step - loss: 0.2283 - acc: 0.9861
Epoch 82/100
25/25 [==============================] - 1s 29ms/step - loss: 0.2195 - acc: 0.9867
Epoch 83/100
25/25 [==============================] - 1s 29ms/step - loss: 0.2141 - acc: 0.9867
Epoch 84/100
25/25 [==============================] - 1s 28ms/step - loss: 0.2104 - acc: 0.9877
Epoch 85/100
25/25 [==============================] - 1s 28ms/step - loss: 0.3928 - acc: 0.9489
Epoch 86/100
25/25 [==============================] - 1s 28ms/step - loss: 0.2756 - acc: 0.9746
Epoch 87/100
25/25 [==============================] - 1s 28ms/step - loss: 0.3193 - acc: 0.9620
Epoch 88/100
25/25 [==============================] -

...

25/25 [==============================] - 1s 29ms/step - loss: 0.2311 - acc: 0.9850
Epoch 51/100
25/25 [==============================] - 1s 29ms/step - loss: 0.2227 - acc: 0.9861
Epoch 52/100
25/25 [==============================] - 1s 29ms/step - loss: 0.3866 - acc: 0.9667
Epoch 53/100
25/25 [==============================] - 1s 29ms/step - loss: 0.4722 - acc: 0.9397
Epoch 54/100
25/25 [==============================] - 1s 30ms/step - loss: 1.6384 - acc: 0.7884
Epoch 55/100
25/25 [==============================] - 1s 29ms/step - loss: 0.5803 - acc: 0.8698
Epoch 56/100
25/25 [==============================] - 1s 30ms/step - loss: 0.4464 - acc: 0.9361
Epoch 57/100
25/25 [==============================] - 1s 29ms/step - loss: 0.3715 - acc: 0.9578
Epoch 58/100
25/25 [==============================] - 1s 29ms/step - loss: 0.3308 - acc: 0.9670
Epoch 59/100
25/25 [==============================] - 1s 29ms/step - loss: 0.3208 - acc: 0.9696
Epoch 60/100
25/25 [==============================] -

25/25 [==============================] - 1s 29ms/step - loss: 0.3451 - acc: 0.9581
Epoch 33/100
25/25 [==============================] - 1s 28ms/step - loss: 0.3429 - acc: 0.9544
Epoch 34/100
25/25 [==============================] - 1s 28ms/step - loss: 0.3385 - acc: 0.9563
Epoch 35/100
25/25 [==============================] - 1s 29ms/step - loss: 0.4258 - acc: 0.9389
Epoch 36/100
25/25 [==============================] - 1s 29ms/step - loss: 0.3964 - acc: 0.9405
Epoch 37/100
25/25 [==============================] - 1s 30ms/step - loss: 0.4175 - acc: 0.9450
Epoch 38/100
25/25 [==============================] - 1s 29ms/step - loss: 0.7388 - acc: 0.8845
Epoch 39/100
25/25 [==============================] - 1s 29ms/step - loss: 0.4397 - acc: 0.9309
Epoch 40/100
25/25 [==============================] - 1s 29ms/step - loss: 0.4053 - acc: 0.9490
Epoch 41/100
25/25 [==============================] - 1s 29ms/step - loss: 0.3388 - acc: 0.9626
Epoch 42/100
25/25 [==============================] -

25/25 [==============================] - 1s 28ms/step - loss: 0.5354 - acc: 0.8471
Epoch 15/100
25/25 [==============================] - 1s 29ms/step - loss: 0.4885 - acc: 0.8742
Epoch 16/100
25/25 [==============================] - 1s 29ms/step - loss: 0.4679 - acc: 0.8895
Epoch 17/100
25/25 [==============================] - 1s 29ms/step - loss: 0.4546 - acc: 0.8963
Epoch 18/100
25/25 [==============================] - 1s 29ms/step - loss: 0.4741 - acc: 0.8965
Epoch 19/100
25/25 [==============================] - 1s 29ms/step - loss: 0.4520 - acc: 0.9088
Epoch 20/100
25/25 [==============================] - 1s 28ms/step - loss: 0.4145 - acc: 0.9185
Epoch 21/100
25/25 [==============================] - 1s 28ms/step - loss: 0.4319 - acc: 0.9240
Epoch 22/100
25/25 [==============================] - 1s 29ms/step - loss: 0.7275 - acc: 0.8225
Epoch 23/100
25/25 [==============================] - 1s 29ms/step - loss: 0.5804 - acc: 0.8153
Epoch 24/100
25/25 [==============================] -

25/25 [==============================] - 1s 27ms/step - loss: 0.2219 - acc: 0.9861
Epoch 100/100
25/25 [==============================] - 1s 26ms/step - loss: 0.2623 - acc: 0.9769
Epoch 1/100
25/25 [==============================] - 2s 28ms/step - loss: 1.9337 - acc: 0.5089
Epoch 2/100
25/25 [==============================] - 1s 28ms/step - loss: 1.0133 - acc: 0.5987
Epoch 3/100
25/25 [==============================] - 1s 28ms/step - loss: 0.8859 - acc: 0.6457
Epoch 4/100
25/25 [==============================] - 1s 28ms/step - loss: 0.8192 - acc: 0.6957
Epoch 5/100
25/25 [==============================] - 1s 28ms/step - loss: 0.7232 - acc: 0.7688
Epoch 6/100
25/25 [==============================] - 1s 29ms/step - loss: 0.6847 - acc: 0.7871
Epoch 7/100
25/25 [==============================] - 1s 29ms/step - loss: 0.7018 - acc: 0.7900
Epoch 8/100
25/25 [==============================] - 1s 28ms/step - loss: 0.7894 - acc: 0.6876
Epoch 9/100
25/25 [==============================] - 1s 28ms

25/25 [==============================] - 1s 28ms/step - loss: 0.2210 - acc: 0.9864
Epoch 82/100
25/25 [==============================] - 1s 29ms/step - loss: 0.2177 - acc: 0.9884
Epoch 83/100
25/25 [==============================] - 1s 28ms/step - loss: 0.2168 - acc: 0.9872
Epoch 84/100
25/25 [==============================] - 1s 28ms/step - loss: 0.2306 - acc: 0.9840
Epoch 85/100
25/25 [==============================] - 1s 28ms/step - loss: 0.2527 - acc: 0.9783
Epoch 86/100
25/25 [==============================] - 1s 28ms/step - loss: 0.2763 - acc: 0.9749
Epoch 87/100
25/25 [==============================] - 1s 28ms/step - loss: 1.5752 - acc: 0.8462
Epoch 88/100
25/25 [==============================] - 1s 28ms/step - loss: 0.8542 - acc: 0.8373
Epoch 89/100
25/25 [==============================] - 1s 28ms/step - loss: 0.6446 - acc: 0.8596
Epoch 90/100
25/25 [==============================] - 1s 28ms/step - loss: 0.5209 - acc: 0.9012
Epoch 91/100
25/25 [==============================] -

In [16]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
preds = final_model.predict(X_test)

precision = precision_score(y_test, preds)
recall = recall_score(y_test, preds)
f1 = f1_score(y_test, preds)
roc_auc = roc_auc_score(y_test, preds)
acc = accuracy_score(y_test, preds)

print(f' accuracy : {acc}, precision : {precision}, recall : {recall}, f1 : {f1}, roc_auc : {roc_auc}')

 accuracy : 0.7488673139158576, precision : 0.7477064220183486, recall : 0.8782327586206896, f1 : 0.8077304261645193, roc_auc : 0.7162638671547533


**예전 LSTM 구조 acc**
- `KerasClassifier(build_fn=lambda:get_model(), epochs=20, batch_size=256)`
  - 1개 : 0.7223300970873786
  - 5개 : 0.7436893203883496
  - 15개 : 0.7372168284789644
  - 25개 : 0.7404530744336569
  - 50개 : 0.7411003236245954
- `KerasClassifier(build_fn=lambda:get_model(), epochs=100, batch_size=256)`
  - 1개 : 0.7313915857605178
  - 5개 : 0.7449838187702266
  - 15개 : { accuracy : 0.7585760517799353, precision : 0.7707317073170732, recall : 0.8512931034482759, f1 : 0.8090117767537122, roc_auc : 0.7352089504275415 }
  - 25개 : { accuracy : 0.7592233009708738, precision : 0.7796780684104627, recall : 0.8351293103448276, f1 : 0.8064516129032258, roc_auc : 0.7400930182194154 }
  - 50개 : accuracy : 0.7488673139158576, precision : 0.7477064220183486, recall : 0.8782327586206896, f1 : 0.8077304261645193, roc_auc : 0.7162638671547533